<a href="https://colab.research.google.com/github/EriveltoViana/EriveltoViana/blob/main/C%C3%B3digo_para_Download_de_V%C3%ADdeos_do_Youtube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pytube

In [ ]:
# Primeiro, vamos instalar as bibliotecas necessárias
!pip install pytube

import os
from pytube import Playlist, YouTube
from google.colab import files
import re
import time

def limpar_nome_arquivo(nome):
    """Remove caracteres inválidos de nomes de arquivos."""
    return re.sub(r'[\\/*?:"<>|]', "", nome)

def baixar_playlist(url_playlist, formato='mp4', apenas_audio=False, qualidade='highest', diretorio='downloads'):
    """
    Função para baixar vídeos de uma playlist do YouTube.

    Parâmetros:
    - url_playlist: URL da playlist do YouTube
    - formato: Formato de saída ('mp4' ou 'mp3')
    - apenas_audio: Baixar apenas o áudio (True/False)
    - qualidade: Qualidade do vídeo ('highest', 'lowest', '720p', etc.)
    - diretorio: Pasta para salvar os downloads
    """
    try:
        # Criar diretório se não existir
        if not os.path.exists(diretorio):
            os.makedirs(diretorio)

        # Obter a playlist
        playlist = Playlist(url_playlist)
        print(f'Playlist encontrada: {playlist.title}')
        print(f'Total de vídeos: {len(playlist.video_urls)}')

        # Contadores para estatísticas
        baixados = 0
        falhas = 0

        # Percorrer cada vídeo na playlist
        for index, url in enumerate(playlist.video_urls):
            try:
                # Obter o objeto YouTube
                yt = YouTube(url)

                # Limpar o título para usar como nome do arquivo
                titulo_limpo = limpar_nome_arquivo(yt.title)
                print(f'\nBaixando ({index+1}/{len(playlist.video_urls)}): {yt.title}')

                if apenas_audio or formato == 'mp3':
                    # Baixar apenas áudio
                    stream = yt.streams.filter(only_audio=True).first()
                    arquivo_saida = os.path.join(diretorio, f'{titulo_limpo}.mp3')
                    out_file = stream.download(output_path=diretorio, filename=f'{titulo_limpo}.mp3')
                else:
                    # Baixar vídeo
                    if qualidade == 'highest':
                        stream = yt.streams.filter(progressive=True).get_highest_resolution()
                    elif qualidade == 'lowest':
                        stream = yt.streams.filter(progressive=True).get_lowest_resolution()
                    else:
                        stream = yt.streams.filter(res=qualidade, progressive=True).first()
                        if not stream:
                            print(f"Qualidade {qualidade} não disponível. Usando a melhor qualidade disponível.")
                            stream = yt.streams.filter(progressive=True).get_highest_resolution()

                    arquivo_saida = os.path.join(diretorio, f'{titulo_limpo}.{formato}')
                    out_file = stream.download(output_path=diretorio, filename=f'{titulo_limpo}.{formato}')

                print(f'✓ Download completo: {arquivo_saida}')
                baixados += 1

                # Pequeno delay para evitar sobrecarga na API
                time.sleep(1)

            except Exception as e:
                print(f'✗ Erro ao baixar {url}: {str(e)}')
                falhas += 1

        print(f'\nResumo do download:')
        print(f'- Total de vídeos na playlist: {len(playlist.video_urls)}')
        print(f'- Vídeos baixados com sucesso: {baixados}')
        print(f'- Falhas: {falhas}')

        # Comprimir todos os arquivos baixados em um único arquivo para facilitar o download do Colab
        import shutil
        arquivo_zip = f'{diretorio}.zip'
        shutil.make_archive(diretorio, 'zip', diretorio)
        print(f'\nArquivos compactados em {arquivo_zip}')

        # Fazer o download do arquivo zip para o computador local
        print('Iniciando download do arquivo zip para seu computador...')
        files.download(f'{arquivo_zip}')

    except Exception as e:
        print(f'Erro ao processar a playlist: {str(e)}')

# Interface simples para o usuário
url = input("Cole a URL da playlist do YouTube: ")
formato = input("Digite o formato desejado (mp4 ou mp3): ").lower() or 'mp4'
apenas_audio = input("Baixar apenas áudio? (s/n): ").lower() == 's'
qualidade = input("Qualidade desejada (highest, lowest, 720p, 480p, 360p, etc.): ") or 'highest'

print("\nIniciando download da playlist...")
baixar_playlist(url, formato, apenas_audio, qualidade)